Fitur yang dipakai :
- TFIDF Word
- WH word
- Question Length

In [1]:
import pandas as pd
import numpy as np
import csv
from nltk.tokenize import word_tokenize
import pickle
from sklearn import preprocessing
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()
import re 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
def wh_tag(doc):
    wh_tag_list = {'apa': 1, 'apakah': 2, 'dimana': 3, 'di mana': 3, 'bagaimana':4, 'bagaimanakah':4, 'mengapa':5, 'kenapa':6, 'siapa':7, 'siapakah':7, 'kapan':8, 'kemana':9, 'dari mana':10, 'berapa':11, 'berapakah':11, 'mana':12, 'manakah':12, 'dimanakah':13}
    result = []
    wh_tags = []
    for t in word_tokenize(doc):
        if t in wh_tag_list:
            wh_tags.append(wh_tag_list[t])
    if not wh_tags:#If empty
        result.append(0) #none tag
    else:
        result.append(wh_tags[0])
    return result

In [35]:
sen_class = 1

target = 'dataset-question-classification-csv.csv'
with open(target,'r',encoding='utf-8') as csvFile:
    csvReader = csv.reader(csvFile, delimiter=';')
    next(csvReader)
    
    text = []
    classes = []
    tag_wh_new = []
    question_length = []
    
    for row in csvReader:
        
        sentence = row[0].lower()
        
        if sen_class == 1:
            sentenceclass =row[1] #row[1]=coarse - row[2]=fine
        elif sen_class == 2:
            sentenceclass =row[2] #row[1]=coarse - row[2]=fine
        classes.append(sentenceclass)
        
        #Question length
        q_length = len(re.findall(r'\w+', sentence)) 
        question_length.append(q_length)
        
        #WH Word
        tag_wh_new.append(wh_tag(sentence.lower()))
        
        text.append(sentence)

In [36]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_feature = tfidf_vectorizer.fit_transform(text)
documents_text = tfidf_feature.toarray()
documents_text = documents_text.tolist()



# Insert Question Length
i=0
for xxxxx in documents_text:
    documents_text[i].insert(0,question_length[i])
    i+=1

# Insert WH Tag 
i=0
for xxxxx in documents_text:
    documents_text[i].insert(0,tag_wh_new[i][0])
    i+=1
    
    
    
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

a_train, a_test, b_train, b_test = train_test_split(documents_text, classes, test_size=0.30, random_state=42)

In [37]:
#SVM model Coarse Category
from sklearn import svm
model = svm.SVC(kernel='linear')
model.fit(a_train,b_train)

predictions = model.predict(a_test)
print(classification_report(b_test,predictions,digits=3))
print('\n\n')

              precision    recall  f1-score   support

        ABBR      0.947     0.857     0.900        21
        DESC      0.857     0.750     0.800        48
        ENTY      0.829     0.934     0.878       166
         HUM      0.978     0.891     0.933       101
         LOC      0.883     0.803     0.841        66
         NUM      0.972     0.986     0.979       143

    accuracy                          0.905       545
   macro avg      0.911     0.870     0.889       545
weighted avg      0.908     0.905     0.904       545






In [34]:
#SVM model Coarse Category
from sklearn import svm
model = svm.SVC(kernel='linear')
model.fit(a_train,b_train)

predictions = model.predict(a_test)
print(classification_report(b_test,predictions,digits=3))
print('\n\n')

                precision    recall  f1-score   support

  abbreviation      0.556     1.000     0.714         5
        animal      0.833     0.714     0.769        14
          body      1.000     0.500     0.667         6
          city      1.000     0.750     0.857         8
          code      1.000     0.750     0.857         4
         color      1.000     0.900     0.947        10
         count      0.585     0.886     0.705        35
       country      0.846     0.917     0.880        12
      creation      0.462     0.857     0.600        14
      currency      1.000     0.800     0.889         5
          date      0.933     0.933     0.933        15
    definition      0.350     1.000     0.519         7
   description      0.500     0.375     0.429         8
        dismed      1.000     0.250     0.400        12
      distance      0.917     0.917     0.917        12
     entyother      0.381     0.533     0.444        15
         event      0.214     0.750     0.333  

c:\users\fandy\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
